[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brunomssmelo/Python_em_Auditoria/blob/master/03_Criar_Novos_Campos.ipynb)

# **Análise de Dados aplicada à Auditoria com _Python_**

## **Técnicas de Auditoria Assistidas por Computador - TAACs**

### _Criar Novos Campos_

> Este documento é uma adaptação para Python do <a href="https://colab.research.google.com/drive/1LA57_2XtNi_JWR0A4QSaOPEfNQ7xT1NT">notebook</a> originalmente codificado em R.

In [1]:
# Antes de prosseguirmos, deveremos carregar aa bibliotecas necessárias:
import pandas as pd

In [2]:
# A importação dos dados pode ser realizada com o método read_table():
hood = pd.read_table('./dados/hood.txt',  encoding='latin1', sep='\s+', decimal=',', na_values=' ')

# Podemos visualizar os registros iniciais dos dados importados com o método head():
hood.head()

,Codigo,Quant,PUnit,Valor
0,10001,13,250.0,3250.0
1,10002,12,275.0,3300.0
2,10003,15,270.0,4050.0
3,10004,11,200.0,2200.0
4,10005,10,400.0,4000.0


Podemos obter um "quadro de informações gerais" dos dados importados para o `DataFrame` `hood` por meio de uma chamada ao método `info()`, que pode ser encarado como uma espécie de equivalente em pandas para a função `glimpse()` do pacote tibble do R:  

In [3]:
hood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 4 columns):
Codigo    553 non-null int64
Quant     553 non-null int64
PUnit     553 non-null float64
Valor     553 non-null float64
dtypes: float64(2), int64(2)
memory usage: 17.4 KB


<br>

A criação de novos campos a partir de campos já existentes no conjunto de dados é uma tarefa ao mesmo tempo corriqueira e simples. 
Para a execução deste procedimento vamos utilizar o método `assign()` da classe `DataFrame` definida na biblioteca `pandas`.

Usaremos o conjunto de dados `hood.txt` para ilustrar esta tarefa. 
Este conjunto de dados pode ser encontrado nas páginas 528 a 538 do livro 
[**Auditoria**](https://www.amazon.com.br/Auditoria-William-C-Boynton/dp/8522432449) de William C.Boynton, Raymond N. Johnson e Walter G. Kell, Ed. Atlas, 2002.

Vamos criar um novo campo na base de dados hood chamado `vlrAudit` que será o produto dos campos `Quant` e `PUnit`.  
Depois vamos criar um novo campo (ao qual denominaremos `diff`) com a diferença absoluta entre os campos `Valor` e `vlrAudit` para ver se existe algum erro.

In [4]:
hood = (hood
            .assign(vlrAudit = hood.Quant * hood.PUnit,
                    dif = lambda x: abs(x.Valor -  x.vlrAudit)))
hood.head()

,Codigo,Quant,PUnit,Valor,vlrAudit,dif
0,10001,13,250.0,3250.0,3250.0,0.0
1,10002,12,275.0,3300.0,3300.0,0.0
2,10003,15,270.0,4050.0,4050.0,0.0
3,10004,11,200.0,2200.0,2200.0,0.0
4,10005,10,400.0,4000.0,4000.0,0.0


Com um filtro podemos ver se algum valor da coluna `dif` é diferente de zero. 

> Na realidade, como pequenos erros de arredondamento poderão ocorrer nas operações envolvendo variáveis reais (tipo `float`),
> é boa prática verificar se os valores do campo `dif` são superiores (lembre-se que este campo correponde a um valor absoluto e 
> portanto será sempre positivo) a um determinado valor bem próximo de zero. Neste exemplo usaremos um valor $\epsilon = 1e-6$.  

In [5]:
(hood
    .query('dif > 1e-6'))

,Codigo,Quant,PUnit,Valor,vlrAudit,dif
503,20004,10052,17.85,179428.0,179428.2,0.2


Para efeitos práticos, pode-se dizer não existir problemas na comparação realizada. 
Existe uma diferença de 20 centavos na fatura de número 20004 entre o valor faturado e o calculado com base na quantidade vendida e preço unitário do item.

Agora vamos criar um novo campo chamado `vlrAcum` contendo o valor acumulado, fatura a fatura, do campo `Valor`.

In [6]:
hood = (hood
            .assign(vlrAcum = hood.Valor.cumsum()))

hood.head()

,Codigo,Quant,PUnit,Valor,vlrAudit,dif,vlrAcum
0,10001,13,250.0,3250.0,3250.0,0.0,3250.0
1,10002,12,275.0,3300.0,3300.0,0.0,6550.0
2,10003,15,270.0,4050.0,4050.0,0.0,10600.0
3,10004,11,200.0,2200.0,2200.0,0.0,12800.0
4,10005,10,400.0,4000.0,4000.0,0.0,16800.0


Existem diversas formas de se eliminar colunas em um data frame. 
Aqui, iremos apresentar 2 possibilidades: usando a palavra chave `del` do *Python* ou por meio do método `drop` da calsse `DataFrame` da biblioteca *pandas*.

A exclusão usando `del` somente pode ser feita utilizando-se a notação com colchetes para fazer referência à coluna a ser excluída:

* Isso funciona: `del df['nome_coluna']`
* Isso não funciona: `del df.nome_coluna`

> **Importante:** ao utilizarmos `del`, ao contrário do comportamento padrão no R, não será retornada uma cópia do `DataFrame` sem a coluna indicada para exclusão.
> A coluna será excluída de forma direta (*inplace*) no próprio `DataFrame` original.

Exemplo: suponha que se queira excluir a variável `dif`. Isso pode ser feito da seguinte forma:

In [7]:
del hood['dif']
hood.head()

,Codigo,Quant,PUnit,Valor,vlrAudit,vlrAcum
0,10001,13,250.0,3250.0,3250.0,3250.0
1,10002,12,275.0,3300.0,3300.0,6550.0
2,10003,15,270.0,4050.0,4050.0,10600.0
3,10004,11,200.0,2200.0,2200.0,12800.0
4,10005,10,400.0,4000.0,4000.0,16800.0


Conforme adiantamos, a exclusão de variáveis fazendo-se referência às mesmas pode ser feita utilizano-se o método 
`drop()` da classe `DataFrame` da biblioteca *pandas*.

> **Observações**:
>
> 1 - o comportamento padrão do método `drop`, assim como no R, é a realização das exclusões numa cópia do `DataFrame`
> original, a qual então é retornada. É possível realizar as exclusões de forma direta ao atribuirmos o valor
> `True` ao parâmetro `inplace` na chamada do método.
>
> 2 - o método `drop` pode ser utilizado tanto para excluir linhas (seu comportamento padrão) ou colunas.
> Para excluirmos colunas deveremos chamar o método `drop` com a lista das colunas a serem excluídas e também
> atribuirmos ao parâmetro `axis` o valor 1.
> Uma forma alternativa seria simplesmente chamar o método `drop` atribuindo-se ao parâmetro `columns` a lista
> com o nome das colunas a serem excluidas.

Exemplo: caso se queira excluir as variáveis `vlrAudit` e `vlrAcum` podemos usar o seguinte comando:

In [8]:
# Forma tradicional:
hood = (hood # esta atribuição é necessaria dado que o método drop irá atuar sobre um cópia de hood e não sobre o DataFrame original            
            .drop(['vlrAudit', 'vlrAcum'], axis=1))

# Abaixo uma forma alternativa:
# hood = (hood # esta atribuição é necessaria dado que o método drop irá atuar sobre um cópia de hood e não sobre o DataFrame original            
#             .drop(columns = ['vlrAudit', 'vlrAcum']))

# Exibe as 5 linhas iniciais de hood
hood.head()

,Codigo,Quant,PUnit,Valor
0,10001,13,250.0,3250.0
1,10002,12,275.0,3300.0
2,10003,15,270.0,4050.0
3,10004,11,200.0,2200.0
4,10005,10,400.0,4000.0


<br>

---

![](./imagens/sua_vez_fundo_transp.png) **Agora é com você...**

---

<br>

**1** - Importe o conjunto de dados `RH.csv`. Calcule uma nova coluna que indique quantos por cento o valor de bônus representa do salário.

<br>

**2** - XXX

<br>

**3** - XXX

<br>

**4** XXX
